In [ ]:
%load_ext autoreload
%autoreload 2

# White background for matplotlib plots
# %config InlineBackend.print_figure_kwargs={'facecolor': "w"}

import numpy as np
import tensorflow as tf
import torch
import tqdm.notebook as tqdm
import pandas as pd

import thesis.quantization_benchmark as qb

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]

import seaborn as sns

# %matplotlib widget

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0520_dense.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
#     ci="sd",
)
g.set(title="Dense", yscale="log")

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0520_dense_3_layer.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
#     ci="sd",
)
g.set(title="Dense - 3 layers", yscale="log")

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0520_cnn_1_layer.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
#     ci="sd",
)
g.set(title="Dense", yscale="log")

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0520_cnn_3_layer.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
#     ci="sd",
)
g.set(title="CNN 3 layers", yscale="log")

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0520_dilated_cnn_small.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
#     ci="sd",
)
g.set(title="Dense", yscale="log")

In [ ]:
df

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0520_dilated_cnn_1_layer.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
#     ci="sd",
)
g.set(title="Dilated CNN - 1 layer", yscale="log")

In [ ]:
df = pd.read_csv("/Volumes/euler/benchmark_data/0520_dilated_cnn_3_layers.csv")
df = df.drop(columns={"Unnamed: 0"})

g = sns.relplot(
    data=df,
    hue="name",
    x="network_size",
    y="inference_time",
    kind="line",
#     ci="sd",
)
g.set(title="Dilated CNN - 3 layers", yscale="log")

## ----

In [ ]:
import pandas as pd
import seaborn as sns

df = pd.concat(runs_per_size)
# df.to_csv("../data/benchmarks/0518-dense-timing-2.csv")

In [ ]:
times = df.groupby(["name", "network_size"]).agg({"inference_time": "mean"})
times["relative_time"] = 0.0

for _, row in times.iterrows():
    row["relative_time"] = row["inference_time"] / times.loc[("ONNXRuntime", row.name[1]), "inference_time"]

times = times.reset_index()
times

In [ ]:
df

In [ ]:
runs_per_size = []
n_sizes = 4

for i in tqdm.trange(n_sizes):
    size = 8 * 2 ** i
    n_channels = 64

    cnn_keras = tf.keras.Sequential(
        [
            tf.keras.layers.Input((size, size, 3)),
            tf.keras.layers.Conv2D(filters=n_channels, kernel_size=3, activation=tf.nn.relu),
            tf.keras.layers.Conv2D(filters=n_channels, kernel_size=3, activation=tf.nn.relu),
            tf.keras.layers.MaxPooling2D(size - 4),
        ]
    )

    cnn_torch = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=3, out_channels=n_channels, kernel_size=3),
        torch.nn.ReLU(),
        torch.nn.Conv2d(in_channels=n_channels, out_channels=n_channels, kernel_size=3),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(size - 4),
    )

    runtimes = qb.get_runtimes(good_only=True, unsigned_weights=True)

    runs = qb.benchmark(cnn_keras, cnn_torch, runtimes, n_iterations=500)
    runs["network_size"] = i
    runs_per_size.append(runs)

In [ ]:
df_cnn = pd.concat(runs_per_size)

In [ ]:
df_cnn.to_csv("../data/benchmarks/0518-cnn-timing-2.csv")

In [ ]:
times_cnn = df_cnn.groupby(["name", "network_size"]).agg({"inference_time": "mean"})
times_cnn["relative_time"] = 0.0

for _, row in times_cnn.iterrows():
    row["relative_time"] = row["inference_time"] / times_cnn.loc[("ONNXRuntime", row.name[1]), "inference_time"]

times_cnn = times_cnn.reset_index()
times_cnn

In [ ]:
g = sns.relplot(data=times_cnn, hue="name", x="network_size", y="relative_time", kind="line")
# g.set(yscale="log")
g.set(title="CNN")

In [ ]:
import seaborn as sns

groups = ["ONNXRuntime", "OpenVINO", "TFLite", "TorchScript"]

for group in groups:
    filtered = times_cnn
    # filtered = filtered.loc[times["inference_time"] < 0.005]
    filtered = filtered.loc[filtered["name"].str.startswith(group)]
    filtered = filtered.loc[~filtered["name"].str.contains("_openvino")]
    filtered = filtered.loc[~filtered["name"].str.contains("static_qoperator")]

    sns.relplot(data=filtered, hue="name", x="network_size", y="relative_time", kind="line")
    plt.title(group)

In [ ]:
len(runs_per_size)

In [ ]:
import tensorflow as tf
import ddsp.training

In [ ]:
dec = ddsp.training.nn.DilatedConvStack(
    ch=256,
    layers_per_stack=5,
    stacks=1,
    kernel_size=3,
    dilation=2,
)

model = tf.keras.Sequential([
    tf.keras.layers.Input((64000, 1, 1)),
    dec,
])

# .decoders.DilatedConvDecoder(
#     ch=256,
#     kernel_size=3,
#     layers_per_stack=5,
#     stacks=1
# )

In [ ]:
model(np.ones((1, 64000, 1, 1)))

In [ ]:
model.input.shape